In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [1]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
import csv
from PIL import Image, ImageFilter    
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
import tensorflow as tf
import numpy as np 
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
import collections
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 


import os
import sys
import argparse
import requests
from tqdm.auto import tqdm 
import time

In [5]:
file_path = './data/telemoji/train/angry' # 각 감정 데이터가 있는 폴더경로를를 설정
file_names = os.listdir(file_path)

i = 1
for name in tqdm(file_names):
    src = os.path.join(file_path, name)
    dst = 'angry.' + str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

  0%|          | 0/3423 [00:00<?, ?it/s]

In [6]:
file_path = './data/telemoji/train/disgust' # 각 감정 데이터가 있는 폴더경로를를 설정
file_names = os.listdir(file_path)

i = 1
for name in tqdm(file_names):
    src = os.path.join(file_path, name)
    dst = 'disgust.' + str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

  0%|          | 0/3423 [00:00<?, ?it/s]

In [7]:
file_path = './data/telemoji/train/fear' # 각 감정 데이터가 있는 폴더경로를를 설정
file_names = os.listdir(file_path)

i = 1
for name in tqdm(file_names):
    src = os.path.join(file_path, name)
    dst = 'fear.' + str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

  0%|          | 0/3423 [00:00<?, ?it/s]

In [8]:
file_path = './data/telemoji/train/happy' # 각 감정 데이터가 있는 폴더경로를를 설정
file_names = os.listdir(file_path)

i = 1
for name in tqdm(file_names):
    src = os.path.join(file_path, name)
    dst = 'happy.' + str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

  0%|          | 0/3423 [00:00<?, ?it/s]

In [9]:
file_path = './data/telemoji/train/neutral' # 각 감정 데이터가 있는 폴더경로를를 설정
file_names = os.listdir(file_path)

i = 1
for name in tqdm(file_names):
    src = os.path.join(file_path, name)
    dst = 'neutral.' + str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

  0%|          | 0/3423 [00:00<?, ?it/s]

In [10]:
file_path = './data/telemoji/train/suprise' # 각 감정 데이터가 있는 폴더경로를를 설정
file_names = os.listdir(file_path)

i = 1
for name in tqdm(file_names):
    src = os.path.join(file_path, name)
    dst = 'surprise.' + str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

  0%|          | 0/3423 [00:00<?, ?it/s]

In [3]:
## Kakao API
# Request



API_URL = 'https://dapi.kakao.com/v2/vision/face/detect'
#MYAPP_KEY = 'ed5af1831c75a7daf1c35ff49c6bf2ab' # 나
#MYAPP_KEY = 'ea80b4247c0b7295489c02a8501665d6' # 엄마
MYAPP_KEY = 'c1c91e71e21bbee8cdc83ab7f30cbc51' 

def detect_face(filename):
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}

    try:
        files = { 'image' : open(filename, 'rb')}
        resp = requests.post(API_URL, headers=headers, files=files)
        resp.raise_for_status()
        return resp.json()
    
    except Exception as e:
        print(str(e))
        sys.exit(0)
    

def feature(filename, detection_result):

    image = Image.open(filename)
    
    faces = detection_result['result']['faces'][0]
    facial_points = faces['facial_points']
    fig_w, fig_h = detection_result['result']['width'], detection_result['result']['height']

    fig,ax = plt.subplots(figsize=(10,10))
    
    target_obj = ['right_eyebrow', 'left_eyebrow', 'jaw', 'right_eye', 'left_eye', 'lip', 'nose']
    
    for each_obj in target_obj:
        for each in facial_points[each_obj]:
            rect_face = patches.Circle((each[0]*fig_w, each[1]*fig_h), linewidth=3)
            ax.add_patch(rect_face)

    ax.imshow(image)
    #plt.show()
    fig_2 = plt.gcf() 
    #plt.show()
    fig_2.savefig(str(filename).replace('jpg','png').replace('train','train_kakao'))
    plt.close(fig_2)

In [ ]:
'''
path_dir = '/content/drive/MyDrive/telemoji/train'
file_list = os.listdir(path_dir) # ['disgust', 'angry', 'fear', 'happy', 'sad', 'neutral', 'suprise']

for fname in file_list:
    temp_dir = os.path.join(path_dir, fname)
#    print(temp_dir)

    files = [f for f in listdir(temp_dir) if isfile(join(temp_dir, f))]
    for i in range(len(files)):
      img_url = str(temp_dir) + '/' +files[i]
      detection_result = detect_face(img_url)
  #feature_result = feature(img_url, detection_result)
'''

In [4]:
path_dir = './data/telemoji/train/neutral'
files = os.listdir(path_dir)
#print(files)
empty_list = []

for fname in tqdm(files):
    img_dir = os.path.join(path_dir, fname)
    detection_result = detect_face(img_dir)
      #print(detection_result)
    if not detection_result['result']['faces']:
      empty_list.append(fname)
      #print('This is empty!')
    else:
      feature_result = feature(img_dir, detection_result)

  0%|          | 0/3423 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



400 Client Error: Bad Request for url: https://dapi.kakao.com/v2/vision/face/detect
Traceback (most recent call last):
  File "<ipython-input-3-8f136659c104>", line 17, in detect_face
    resp.raise_for_status()
  File "/usr/local/lib/python3.6/dist-packages/requests/models.py", line 943, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://dapi.kakao.com/v2/vision/face/detect

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-4f4f8fee953c>", line 8, in <module>
    detection_result = detect_face(img_dir)
  File "<ipython-input-3-8f136659c104>", line 22, in detect_face
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception o

TypeError: object of type 'NoneType' has no len()

In [5]:
print(len(empty_list))

164
